In [1]:
import os
import sys
from six.moves import xrange
import tensorflow as tf

In [25]:
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'
IMAGE_SIZE = 24
batch_size = 128
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000

In [3]:
filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in xrange(1, 6)]
filename_queue = tf.train.string_input_producer(filenames)

In [22]:
def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch([image, label], batch_size=batch_size, num_threads=num_preprocess_threads, capacity=min_queue_examples + 3 * batch_size, min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch([image, label], batch_size=batch_size, num_threads=num_preprocess_threads, capacity=min_queue_examples + 3 * batch_size)
    tf.summary.image('images', images)
    return images, tf.reshape(label_batch, [batch_size])

In [4]:
class CIFAR10Record(object):                                                
    pass
result = CIFAR10Record() 

In [7]:
label_bytes = 1
result.height = 32
result.width = 32
result.depth = 3 #R,G,B 3 channels
image_bytes = result.height * result.width * result.depth
record_bytes = label_bytes + image_bytes
reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
result.key, value = reader.read(filename_queue)
record_bytes = tf.decode_raw(value, tf.uint8)
result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)
depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], [label_bytes + image_bytes]), [result.depth, result.height, result.width])
result.uint8image = tf.transpose(depth_major, [1, 2, 0])
reshaped_image = tf.cast(result.uint8image, tf.float32)

In [10]:
height = IMAGE_SIZE
width = IMAGE_SIZE

In [19]:
distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
distorted_image = tf.image.random_flip_left_right(distorted_image)
distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)
float_image = tf.image.per_image_standardization(distorted_image)
float_image.set_shape([height, width, 3])
result.label.set_shape([1])

In [21]:
min_fraction_of_examples_in_queue = 0.4
min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
print ('Filling queue with %d CIFAR images before starting to train. This will take a few minutes.' % min_queue_examples)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [26]:
_generate_image_and_label_batch(float_image, result.label, min_queue_examples, batch_size, shuffle=True)

(<tf.Tensor 'shuffle_batch:0' shape=(128, 24, 24, 3) dtype=float32>,
 <tf.Tensor 'Reshape_2:0' shape=(128,) dtype=int32>)